# Feature Selection and Eng for Logistic Regression Model

In [50]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from sklearn import preprocessing, pipeline, metrics, model_selection
from sklearn.model_selection import GridSearchCV
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100
sns.set_style("white")
%matplotlib inline

### concat table_a (2011) and table_b (2013)

In [2]:
a_df = pd.read_csv("LoanStats3a_securev1.csv",engine="python",error_bad_lines=False,header=1)
b_df = pd.read_csv("LoanStats3b_securev1.csv",engine="python",error_bad_lines=False,header=1)
a_df['table'] = 'a'
b_df['table'] = 'b'
full_concat_df = pd.concat([a_df, b_df])

Skipping line 3357: unexpected end of data


### Features not selected if fully missing in table_a and table_b

all missing val faetures in combined table_a and table_b

In [3]:
concat_full_null_list = ['member_id',  'next_pymnt_d',  'annual_inc_joint',  'dti_joint',  'verification_status_joint', 
                           'open_acc_6m',  'open_act_il',  'open_il_12m',  'open_il_24m',  'mths_since_rcnt_il',  
                           'total_bal_il',  'il_util',  'open_rv_12m',  'open_rv_24m',  'max_bal_bc',  'all_util',  'inq_fi',  
                           'total_cu_tl',  'inq_last_12m',  'revol_bal_joint',  'sec_app_fico_range_low',  'sec_app_fico_range_high',  
                           'sec_app_earliest_cr_line',  'sec_app_inq_last_6mths',  'sec_app_mort_acc',  'sec_app_open_acc',  'sec_app_revol_util', 
                           'sec_app_open_act_il',  'sec_app_num_rev_accts',  'sec_app_chargeoff_within_12_mths',  'sec_app_collections_12_mths_ex_med',  
                           'sec_app_mths_since_last_major_derog']

all missing val features in table_a, i.e. year 2011 data

In [4]:
a_full_null_list = ['tot_coll_amt',  'tot_cur_bal',  'total_rev_hi_lim',  'acc_open_past_24mths',  'avg_cur_bal',  'mo_sin_old_rev_tl_op',  
                    'mo_sin_rcnt_rev_tl_op',  'mo_sin_rcnt_tl',  'mort_acc',  'num_accts_ever_120_pd',  'num_actv_bc_tl',  'num_actv_rev_tl',  
                    'num_bc_sats',  'num_bc_tl',  'num_il_tl',  'num_op_rev_tl',  'num_rev_accts',  'num_rev_tl_bal_gt_0',  'num_sats',  'num_tl_30dpd', 
                    'num_tl_90g_dpd_24m',  'num_tl_op_past_12m',  'pct_tl_nvr_dlq',  'tot_hi_cred_lim',  'total_bal_ex_mort',  
                    'total_bc_limit',  'total_il_high_credit_limit']

all missing val feature in table_b, year 2013 data

In [5]:
b_full_null_list = ['revol_util']

__features left after remove all those columns above__

In [6]:
concat_selected_column_list  = list(set(full_concat_df.columns) - set(concat_full_null_list + a_full_null_list + b_full_null_list))

### Application info 
Those features are possibly required when applying for a loan

In [9]:
application_features = ['id','addr_state', 'annual_inc','annual_inc_joint','application_type', 'desc', 'dti', 'dti_joint', 'earliest_cr_line', 
'emp_title', 'emp_length', 'emp_title','fico_range_high', 'fico_range_low','home_ownership', 'loan_amnt', 'msa', 
'pub_rec_bankruptcies', 'pub_rec', 'purpose', 'tax_liens','term', 'title', 'verification_status',
'verified_status_joint','zip_code', 'sec_app_fico_range_high' , 'sec_app_fico_range_low']

__filtering the features of all null vals__

In [11]:
filtered_apply_features = list(set(concat_selected_column_list).intersection(set(application_features)))

In [12]:
filtered_apply_features = filtered_apply_features + ['issue_d','table']

__features info__

Use code below to view the features's missing value info
```python
full_concat_df[filtered_apply_features].info()

full_concat_df[filtered_apply_features].isnull().sum()
```

- 'desc' has the higheset num of missing vals 3726
- 'emp_title' has 345 missing vals

## Data Values Check

Code before for feature sanity check, 

```python
full_concat_df['pub_rec'].value_counts()
sns.barplot(x='pub_rec', y = 'id', data=full_concat_df, hue='loan_status', estimator=np.ma.count)
full_concat_df['fico_range_high'].value_counts()
full_concat_df['fico_range_low'].value_counts()
sns.scatterplot(x='fico_range_low', y='fico_range_high', data=full_concat_df,hue='loan_status')
full_concat_df['loan_amnt'].value_counts()
full_concat_df['annual_inc'].value_counts()
full_concat_df['verification_status'].value_counts()
full_concat_df['issue_d'].value_counts()
full_concat_df['earliest_cr_line'].value_counts()
full_concat_df['home_ownership'].value_counts()
full_concat_df['title'].value_counts()
full_concat_df['purpose'].value_counts()
full_concat_df['dti'].value_counts()
full_concat_df['term'].value_counts()
full_concat_df['purpose'].value_counts()
full_concat_df['application_type'].value_counts()
full_concat_df['pub_rec_bankruptcies'].value_counts()
full_concat_df['zip_code'].value_counts()
full_concat_df['tax_liens'].value_counts()
full_concat_df['emp_title'].value_counts()
full_concat_df['desc'].value_counts()
```

After initial check the values of each feature, "purpose， dti, term,  loan_amnt, pub_rec_bankruptcies, zip_code, tax_liens, emp_title" are selected

In [15]:
final_selected_features = ['id', 'issue_d','purpose',  'dti',  'term',  'loan_amnt',  'annual_inc',  'pub_rec_bankruptcies',  
 'zip_code',  'tax_liens', 'emp_length', 'emp_title' , 'verification_status', 'earliest_cr_line',  'home_ownership' ,'fico_range_low', 'fico_range_high']

## Data Transformation and Exploration

'loan_income_ration' for ratio of loan amount to the annuual incomde provided by borrower

In [16]:
full_concat_df['loan_income_ratio'] = full_concat_df['loan_amnt'] / full_concat_df['annual_inc']

"issue_d" and "earliest_cr_line" features has no missing val and encoded to date format

In [17]:
datetime_vars = ['issue_d', 'earliest_cr_line']

In [18]:
dt_encode_no_missing_vars = []
for dt_no_missing_var in datetime_vars:
    print(f"datetime encode: {dt_no_missing_var}")
    y_m_encode_var = dt_no_missing_var + "_Y-M"
    dt_encode_no_missing_vars.append(y_m_encode_var)
    y_encode_var = dt_no_missing_var + "_year"
    dt_encode_no_missing_vars.append(y_encode_var)
    m_encode_var = dt_no_missing_var + "_month"
    dt_encode_no_missing_vars.append(m_encode_var)
    
    full_concat_df[y_m_encode_var] = pd.to_datetime(full_concat_df[dt_no_missing_var], format="%b-%Y")
    full_concat_df[y_encode_var] = pd.to_datetime(full_concat_df[dt_no_missing_var], format="%b-%Y").apply(lambda x:x.year)
    full_concat_df[m_encode_var] = pd.to_datetime(full_concat_df[dt_no_missing_var], format="%b-%Y").apply(lambda x:x.month)

datetime encode: issue_d
datetime encode: earliest_cr_line


In [19]:
dt_encode_no_missing_vars

['issue_d_Y-M',
 'issue_d_year',
 'issue_d_month',
 'earliest_cr_line_Y-M',
 'earliest_cr_line_year',
 'earliest_cr_line_month']

Encode Categorical Values

"purpose" label encoder 

In [20]:
full_concat_df['purpose'].isnull().sum()

0

In [21]:
LB_purpose = preprocessing.LabelEncoder()

to_le_var = 'purpose'
print ("Label Encoding %s" % (to_le_var))
LE_var=to_le_var+'_le'
full_concat_df[LE_var]=LB_purpose.fit_transform(full_concat_df[to_le_var])

print ("Label-encoded feaures: %s" % (LE_var))

Label Encoding purpose
Label-encoded feaures: purpose_le


In [22]:
LB_purpose.classes_

array(['car', 'credit_card', 'debt_consolidation', 'home_improvement',
       'house', 'major_purchase', 'medical', 'moving', 'other',
       'renewable_energy', 'small_business', 'vacation', 'wedding'],
      dtype=object)

"term" encoding 

In [48]:
full_concat_df['term_encode'] = full_concat_df['term'].apply(lambda x: 0 if '36' in x else 1)

zip_code: truncated zip_code to 'other' when count <20 (roughly estimate by check the value_counts plot. Could be fine-tuned later

In [23]:
other_zipcode_list = full_concat_df['zip_code'].value_counts().index[full_concat_df['zip_code'].value_counts()<20]

In [24]:
def replace_rare_zipcode(val):
    if val in other_zipcode_list:
        return 'other'
    return val

In [25]:
full_concat_df['zip_code_truncated'] = full_concat_df['zip_code'].apply(replace_rare_zipcode)

In [26]:
LB_zip = preprocessing.LabelEncoder()

to_le_var = 'zip_code_truncated'
print ("Label Encoding %s" % (to_le_var))
LE_var=to_le_var+'_le'
full_concat_df[LE_var]=LB_zip.fit_transform(full_concat_df[to_le_var])

print ("Label-encoded feaures: %s" % (LE_var))

Label Encoding zip_code_truncated
Label-encoded feaures: zip_code_truncated_le


In [27]:
full_concat_df['zip_code_truncated_le'].value_counts()

94    3465
84      88
14      87
11      76
50      76
      ... 
52      20
2       20
45      20
88      20
25      20
Name: zip_code_truncated_le, Length: 95, dtype: int64

"emp_title" encoding, truncated emp_title by replacy others when count of that category is less than 10. This truncated number could be fine-tuned later

In [28]:
other_emp_title_list = full_concat_df['emp_title'].value_counts().index[full_concat_df['emp_title'].value_counts()<10]

In [29]:
def replace_rare_emp_title(val):
    if val in other_emp_title_list:
        return 'other'
    return val

In [30]:
full_concat_df['emp_title_truncated'] = full_concat_df['emp_title'].apply(replace_rare_emp_title)

In [31]:
full_concat_df['emp_title_truncated'].value_counts()

other                  5830
Teacher                  86
Manager                  52
RN                       30
Project Manager          27
Engineer                 25
Sales                    24
Supervisor               22
Registered Nurse         21
Director                 16
Accountant               15
Executive Assistant      14
manager                  12
Account Manager          12
General Manager          12
Driver                   12
Vice President           11
mechanic                 10
Executive Director       10
Name: emp_title_truncated, dtype: int64

In [32]:
full_concat_df['emp_title_truncated_impute'] = full_concat_df['emp_title_truncated'].fillna('Missing')
LB_emp_title = preprocessing.LabelEncoder()

to_le_var = 'emp_title_truncated_impute'
print ("Label Encoding %s" % (to_le_var))
LE_var=to_le_var+'_le'
full_concat_df[LE_var]=LB_emp_title.fit_transform(full_concat_df[to_le_var])

print ("Label-encoded feaures: %s" % (LE_var))

Label Encoding emp_title_truncated_impute
Label-encoded feaures: emp_title_truncated_impute_le


"Employment length" encoding, the missing value replaced by -999

In [33]:
full_concat_df['emp_length'].isnull().sum()

216

In [34]:
full_concat_df['emp_length_impute'] = full_concat_df['emp_length'].fillna('-999')
full_concat_df['emp_length_impute_num'] = full_concat_df['emp_length_impute'].str.extract(r"([-+]?[0-9]+)")\
.astype('float').astype('int')

"verification_status" enconding

In [35]:

full_concat_df['verification_status'].isnull().sum()

0

In [36]:
def verfication_status_encode(val):
    if val == 'Verified':
        return 0
    if val == 'Source Verified':
        return 1
    else:
        return 2

In [37]:
full_concat_df['verification_status_le'] = full_concat_df['verification_status'].apply(verfication_status_encode)

"home_ownership" encoding

In [38]:
full_concat_df['home_ownership'].isnull().sum()

0

In [39]:
full_concat_df['home_ownership'].value_counts()

RENT        3065
MORTGAGE    2998
OWN          523
Name: home_ownership, dtype: int64

In [40]:
def home_ownership_encode(val):
    if val == 'RENT':
        return 0
    if val == 'OWN':
        return 1
    else:
        return 2

In [41]:
full_concat_df['home_ownership_le'] = full_concat_df['home_ownership'].apply(home_ownership_encode)

In [42]:
full_concat_df['home_ownership_le'].value_counts()

0    3065
2    2998
1     523
Name: home_ownership_le, dtype: int64

"tax_liens" encoding


In [43]:
full_concat_df['tax_liens'].isnull().sum()

0

In [44]:
full_concat_df['tax_liens'].value_counts()

0    6499
1      54
2      21
3       8
4       3
8       1
Name: tax_liens, dtype: int64

In [45]:
LB_tax = preprocessing.LabelEncoder()

to_le_var = 'tax_liens'
print ("Label Encoding %s" % (to_le_var))
LE_var=to_le_var+'_le'
full_concat_df[LE_var]=LB_tax.fit_transform(full_concat_df[to_le_var])

print ("Label-encoded feaures: %s" % (LE_var))

Label Encoding tax_liens
Label-encoded feaures: tax_liens_le


In [46]:
full_concat_df['tax_liens_le'].value_counts()

0    6499
1      54
2      21
3       8
4       3
5       1
Name: tax_liens_le, dtype: int64

output the feature vars after encoding to csv file for random forest model prediction

In [49]:
full_concat_df.to_csv("df_to_RF_model.csv")